<a href="https://colab.research.google.com/github/amannraawat/dog-breed-classification/blob/master/dog_breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Class Dog Breed Classification

This notebook builds and end-to-end multi-class classification using Tensorflow.

## Problem

Identifying the breed of dog from the image.

## Data

The data we are using is from Dog Breed Identification Competition(Kaggle).

https://www.kaggle.com/competitions/dog-breed-identification

In [3]:
# unzip the data into the Google Drive.
# !unzip "drive/My Drive/Dog Vision/dog-breed-identification.zip" -d "drive/My Drive/Dog Vision/"